# ConvNet on MNIST

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np

import sys
sys.path.append('../')
from Feedforward import *

## 1. Load MNIST

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
batch_X, batch_y = mnist.train.next_batch(100)

In [4]:
print(np.reshape(batch_X, [-1,28,28,1]).shape)
print(batch_y.shape)


(100, 28, 28, 1)
(100, 10)


## 2. Model definition

In [5]:
sess = tf.InteractiveSession()

In [6]:
input_size= [28, 28]
n_outputs=  batch_y.shape[1]
n_input_maps= 1

n_filters = [32, 64]   
filter_size = [[5,5], [5,5]] 
pool_size =   [[2,2], [2,2]] 
n_hidden = [1024]

batch_size= 100

In [7]:
# for dropout
drop_prob = tf.placeholder(tf.float32)

# Define ConvNet
conv_net= ConvNet( input_size, n_input_maps, n_outputs, n_filters, filter_size, pool_size, n_hidden, name='ConvNet')

# Define train setup
train= conv_net.setup(batch_size, drop_prob, loss_type= 'cross_entropy')

# Optimizer.
optimizer = tf.train.AdamOptimizer(1e-3).minimize(train.loss) #0.001
  
# Predictions for the training, validation, and test data.
train_prediction = tf.nn.softmax(train.y)

Shape of input matrix entering to Fully connected layers: [100, 4, 4, 64]


## 3. Training

In [8]:
def accuracy(predictions, labels):
    if n_outputs == 1:
        return (100.0 * np.sum(np.greater(predictions, 0.5) == np.greater(labels, 0.5))/ predictions.shape[0])
    else:
        return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))/ predictions.shape[0])

In [9]:
num_steps = 5000
summary_freq= 100
n_test_logg= 10 # number of evaluations on test dataset (for logging information)

tf.initialize_all_variables().run()
print('Initialized')

mean_loss= 0
train_accuracy= 0
for step in range(num_steps):
    batch_X, batch_y= mnist.train.next_batch(batch_size)
    batch_X= np.reshape(batch_X, [-1,28,28,1])
    
    
    feed_dict = {train.inputs : batch_X, train.labels : batch_y, drop_prob:1.0}
    
    _, l, train_pred = sess.run([optimizer, train.loss, train_prediction],feed_dict=feed_dict)
    
    train_accuracy += accuracy(train_pred, batch_y)
    mean_loss += l    
    
    if step%summary_freq == 0:
        # train
        train_accuracy= train_accuracy/summary_freq
        
        # test
        test_accuracy= 0
        for i in range(n_test_logg):
            batch_X_test, batch_y_test= mnist.test.next_batch(batch_size) 
            batch_X_test= np.reshape(batch_X_test, [-1,28,28,1])

            pred = train_prediction.eval(feed_dict={train.inputs: batch_X_test, drop_prob: 1.0})
            
            test_accuracy += accuracy(pred, batch_y_test)
        test_accuracy= test_accuracy/n_test_logg
            
        print(step, ', train:',train_accuracy,' | test:', test_accuracy, ' | loss:', mean_loss/summary_freq)
        mean_loss= 0
        train_accuracy= 0

Initialized
0 , train: 0.15  | test: 15.7  | loss: 0.0491150379181
100 , train: 85.97  | test: 94.7  | loss: 0.546582514569
200 , train: 96.59  | test: 96.0  | loss: 0.110534006078
300 , train: 97.22  | test: 97.3  | loss: 0.0941511325119
400 , train: 97.39  | test: 97.1  | loss: 0.0853816034365
500 , train: 97.45  | test: 99.4  | loss: 0.0786991246324
600 , train: 98.92  | test: 98.8  | loss: 0.0386709438986
700 , train: 98.69  | test: 99.9  | loss: 0.0398223515041
800 , train: 98.64  | test: 99.7  | loss: 0.0431913377217
900 , train: 98.65  | test: 98.7  | loss: 0.0406216167193
1000 , train: 98.58  | test: 98.6  | loss: 0.0427045319567
1100 , train: 99.01  | test: 98.5  | loss: 0.0312645349768
1200 , train: 99.37  | test: 99.0  | loss: 0.0204411710543
1300 , train: 99.36  | test: 99.0  | loss: 0.0190941173793
1400 , train: 99.18  | test: 98.7  | loss: 0.0240048117004
1500 , train: 99.01  | test: 99.0  | loss: 0.0289342738729
1600 , train: 99.0  | test: 99.4  | loss: 0.032181808129
17